# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases   Deaths    PCases  PDeaths  Cases (+)  \
0               US  95713363  1053840  95456800  1052313     256563   
1            India  44543089   528370  44516479   528250      26610   
2           France  35138509   155783  35015024   155651     123485   
3           Brazil  34592163   685424  34558902   685121      33261   
4          Germany  32740593   149079  32604993   148728     135600   
5      South Korea  24461737    27891  24264470    27665     197267   
6   United Kingdom  23803310   206873  23772019   206751      31291   
7            Italy  22169273   176609  22096450   176404      72823   
8            Japan  20731652    43842  20396918    43207     334734   
9           Russia  20192164   378324  19915400   377823     276764   
10          Turkey  16852382   101068  16852382   101068          0   
11           Spain  13384876   113725  13374116   113279      10760   
12         Vietnam  11460227    43141  11448034    43132      12193   
13       Australia  10161241    14783  10122875    14458      38366   
14       Argentina   9703938   129855   9697763   129830       6175   

    Deaths (+)  Fatality Rate      Continent  
0         1527            1.1  North America  
1          120            1.2           Asia  
2          132            0.4         Europe  
3          303            2.0  South America  
4          351            0.5         Europe  
5          226            0.1           Asia  
6          122            0.9         Europe  
7          205            0.8         Europe  
8          635            0.2           Asia  
9          501            1.9         Europe  
10           0            0.6           Asia  
11         446            0.8         Europe  
12           9            0.4           Asia  
13         325            0.1        Oceania  
14          25            1.3  South America

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2022-09-19 00:00:00'),
 'since': Timestamp('2022-09-14 00:00:00'),
 'Cases': 612134615,
 'Deaths': 6524373,
 'Cases (+)': 2090200,
 'Deaths (+)': 7172,
 'China Cases': 2696527,
 'China Deaths': 15133,
 'China Cases (+)': 43813,
 'China Deaths (+)': 71,
 'US Cases': 95713363,
 'US Deaths': 1053840,
 'US Cases (+)': 256563,
 'US Deaths (+)': 1527,
 'EU Cases': 225907997,
 'EU Deaths': 1930494,
 'EU Cases (+)': 818660,
 'EU Deaths (+)': 2384}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  9/10/22  9/11/22  9/12/22  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...      160      294      257   
Albania               0        0        0  ...      106       88       17   
Algeria               0        0        0  ...       10        7       12   
Andorra               0        0        0  ...        0        0        0   
Angola                0        0        0  ...        0        0        0   

                9/13/22  9/14/22  9/15/22  9/16/22  9/17/22  9/18/22  9/19/22  
Country/Region                                                                 
Afghanistan         222      347      119      122       74      174      194  
Albania             138      104       89       75       81       43       18  
Algeria               0       19       14       15        7        3        3  
Andorra               0       34        0        0        0        0        0  
Angola                0        0        0        0        0        0        0  

[5 rows x 972 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)